# Assignment  week 7

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: 

__Student id(s)__ : 

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

# Infoboxes

* We willen voor elke Wikipedia pagina de informatie uit de infobox halen en die gestructureerd opslaan.
* In het simpele geval is de infobox een lijst van attribuut-waarde paren:
    * het zijn attributen van de "entiteit" waar de pagina over gaat.
    * Dit zijn dus typisch _RDF subject-predicate-object triples_
    * Die je in eerste orde logica als een binaire relatie zou opslaan (bijvoorbeeld _DateOfBirth(Albert-Speer,'19 maart 1905')_)
* Je zal zien dat hier flink wat haken en ogen aan zitten.
    



## Inspecteer een voorbeeld

* We gaan eens kijken hoe we de attribuut-waarde paren uit de zogenaamde _infoboxen_ uit de Wikipedia paginas kunnen gaan halen.
* Vergelijk eens het begin van de file `NLWikipedia/small.xml` met de pagina <https://nl.wikipedia.org/wiki/Albert_Speer>
* Die infobox informatie ziet er goed en redelijk eenvoudig te extraheren uit.
* Er zijn verschillende _mediawiki_ (het formaat van wikipedia paginas) parsers in omloop. 

### Je bent niet de eerste met deze opdracht ....

* Zoek op het web.
* Bijvoorbeeld: <http://stackoverflow.com/questions/8088226/content-of-infobox-of-wikipedia/21107068#21107068>
* Zie ook <https://www.mediawiki.org/wiki/Alternative_parsers> voor nog meer mediawiki parsers
* Denk ook eens aan <http://pandoc.org/demos.html>
* Zie ook [WikipediaInfoBoxes](WikipediaInfoBoxes) is de assignments folder.

## Vragen

## Download, en extraheer.  (33%)
* Download de dump met alle NL wikipedia paginas.


1. Verzin een goede datastructuur voor de subject-predicate-object triples die je uit de infoboxes per pagina gaat extraheren.
1. Schrijf een functie die uit het text element het infobox deel extraheert, en die lijst attribuut-waarde paren omzet naar jouw datastructuur.
2. Pas je functie toe op het [voorbeeld](NLWikipedia/sample.xml, en debug je code.

### Nou op de hele file/dump
1. Zoek uit hoe je een file stromend kunt inlezen in Python.
2. Pas nu je code toe op de hele file. 
3. Misschien  moet je wel  steeds gegevens naar een file gaan wegschrijven. 
4. Is jouw gekozen data formaat  handig?

#### Hints:
* Wil niet meteen alles tegelijk. Als er te veel variatie in de data zit, richt je dan op de meest voorkomende gevallen, en probeer die goed te krijgen. 
* Probeer wel een beeld te krijgen van "wat je mist", en beschrijf dat ook. 

## Analyseer  (33%)

1. Maak statistieken, wat zijn de meest gebruikte attributen, hoe vaak worden ze gebruikt, hoeveel attributen zijn er? Welke waardes worden er veel gebruikt? Hoeveel attributen zie je gemiddeld/mediaan per pagina? 
2. Doe een analyse op de waardes van de attributen? Kan je bepaalde types vinden?
3. Doe een analyse op de uniformiteit van de attribuutnamen. Uit de literatuur blijkt dat er erg veel verschillende manieren worden gebruikt om hetzelfde attribuut weer te geven (bijv, sterfdag, sterfdatum, sterftejaar, etc). Is dat ook zo in jouw dataset? Hoe kom je daar eigenlijk achter? 
 
 
## Reflecteer  (33%)
* Je hebt nu flink wat tijd besteed aan wat de mensen achter DBpedia, Yago, FreeBase en meer van dit soort initiatieven ook al doen. De bedoeling van de opdracht was om te ervaren wat een klus het is om een kennisbank te vullen met "gestructureerde" informatie van het web. 
* Beschrijf puntsgewijs wat volgens jullie de lastige onderdelen zijn van dit proces.
* Licht er 2-3 uit, en vertel hoe jullie het probleem hebben aangepakt.

# Mijn antwoorden

## Inladen van de wikipedia dump in onze structuur

Er is uiteindelijk gekozen voor de "iterparse"  om de wikipedia dump te parsen(dit is verder uitgeleged in de reflectie). De main functie is "extractXML" waarmee de dump word geparsed. We parsen voor een aantal ingegeven minuten de wikipedia dump. Na ongeveer 1 minuut van parsen zijn er ongeveer 6000 attributen in de datastructuur. 

In [2]:
import json
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET
import time

wikiDict = {}

def extractXML(minutes):
    seconds = minutes*60
    start_time = time.time()
    global wikiDict
    parser = ET.iterparse("nlwiki-latest-pages-articles.xml")
    
    # Load the saved datastructure
    wikiDict = importDict()
    print len(wikiDict)
    
    # Main loop for parsing the wiki pages
    for event, element in parser:
        elapsed_time = time.time() - start_time
        
        # If enough time has passed, stop
        if elapsed_time < seconds:
            if "title" in element.tag:
                title = element.text
            elif "text" in element.tag:
                
                # Not in current wikiDict
                if not (title in wikiDict):
                    infobox = getInfobox(element.text)
                    dataObject = getObject(infobox)  
                    if dataObject:
                         wikiDict[title] = dataObject
            element.clear()
        else:
            break
    
    # Print results 
    print len(wikiDict)
    backupDict(wikiDict)
            
def getInfobox(text):
#     text = str(text)
    start = text.find("{{Infobox")
    end = text.find("\n}}\n", start)
    return text[start:end+2]
    
def getObject(infobox):
    wikiObject = {}
    for line in infobox.split("\n"):
        if line[0:1] == "|":
            pair = line[2:].split("=")
            if pair[0].strip() != "" and len(pair) > 1:
                key = pair[0].strip()
                value = pair[1].strip()
                wikiObject[key] = value
            else:
                return False
    return wikiObject
            
def backupDict(wikiDict):
    with open('wikiDict.json', 'w') as fp:
        json.dump(wikiDict, fp)
        
def importDict():
    with open('wikiDict.json', 'r') as fp:
        return json.load(fp)
    
extractXML(1)
    

22
26747


# Analyses

## Attributen informatie inladen

In [5]:
import operator
attributes = {}
    
def inspectAttributes():
    global wikiDict
    for page in wikiDict:
        for attr in wikiDict[page]:
            value = wikiDict[page][attr]
            if attr in attributes:
                attributes[attr]["occurrence"] += 1
                if value in attributes[attr]:
                    attributes[attr][value] += 1
                else:
                    attributes[attr][value] = 1
            else:
                attributes[attr] = {value : 1}
                attributes[attr] = {"occurrence" : 1}
            
#     print attributes
    
inspectAttributes()

## Hoeveel attributen in totaal?

In [6]:
print "Het totaal aantal attributen in de datastructuur is "+str(len(attributes)

SyntaxError: unexpected EOF while parsing (<ipython-input-6-464866b3d315>, line 1)

## Attributen informatie inladen


In [3]:
import pprint
import operator
attributes = {}
    
def inspectAttributes():
    global wikiDict
    for page in wikiDict:
        for attr in wikiDict[page]:
            value = wikiDict[page][attr]
            if attr in attributes:
                attributes[attr]["occurrence"] += 1
#                 if value != "":
                if value in attributes[attr]:
                    attributes[attr][value] += 1
                else:
                    attributes[attr][value] = 1
            else:
                attributes[attr] = {value : 1}
                attributes[attr]['occurrence'] = 1
            
    pprint.pprint(attributes)
    
inspectAttributes()

{u'ATP-id': {u'A092': 1, 'occurrence': 1},
 u'AustralianOpen': {u"'''Winnaar''' (1995, 2000, 2001, 2003)": 1,
                     'occurrence': 1},
 u'CIAkaart': {u'Algeria CIA map.png': 1,
               u'Andorramap.png': 1,
               'occurrence': 2},
 u'DiseasesDB': {u'5938': 1, 'occurrence': 1},
 u'Dubbelhoogstepositie': {u'123e': 1, 'occurrence': 1},
 u'Dubbeloverwinningen': {u'1': 1, 'occurrence': 1},
 u'Dubbelrecord': {u'40-42': 1, 'occurrence': 1},
 u'Enkelhoogstepositie': {u"'''1e'''": 1, 'occurrence': 1},
 u'Enkeloverwinningen': {u'60': 1, 'occurrence': 1},
 u'Enkelrecord': {u'870-274': 1, 'occurrence': 1},
 u'Hall of Fame-idx': {u'andre-agassi-0': 1, 'occurrence': 1},
 u'Hall of Fame-jaar': {u'2011': 1, 'occurrence': 1},
 u'ICD10': {'occurrence': 1, u'{{ICD10|B|24||b|20}}': 1},
 u'ICD9': {'occurrence': 1, u'{{ICD9|042}}': 1},
 u'ICDO': {u'': 1, 'occurrence': 1},
 u'MedlinePlus': {u'000594': 1, 'occurrence': 1},
 u'OMIM': {u'': 1, 'occurrence': 1},
 u'Ontstaan': {u"'''

## Het aantal 

In [10]:
print "Het totaal aantal attributen in de datastructuur is "+str(len(attributes))

Het totaal aantal attributen in de datastructuur is 284


## Aantal keer dat attributen voorkomen(gesorteerd van meest gebruikt naar minst)

In [23]:
def sortAttributes():
#     sortedAttributes = sorted(attributes.items(), key=operator.itemgetter(1), reverse=True)
    sortedAttributes = sorted(attributes, key=lambda x:attributes[x].get('occurrence'), reverse=True)
    count = 0
    for attr in sortedAttributes:
        print attr, 
        print attributes[attr]['occurrence']
#         try:
#             print str(attr[0])+" :\t\t"+str(attr[1]['occurrence'])
#             count += 1
#         except:
#             pass
        



sortAttributes()
# sort()

naam 18
afbeelding 15
onderschrift 11
website 10
religie 8
handtekening 6
geboortedatum 5
portaal 5
sterfplaats 5
geboorteplaats 5
geboren 4
volledige naam 4
wapen 4
overleden 4
nationaliteit 4
volledigenaam 4
beroep 4
vlag 4
sterfdatum 4
bekendvan 4
munteenheid 3
regeringsvorm 3
volkslied 3
pctwater 3
talen 3
functie1 3
functie2 3
functie3 3
tel 3
Ontstaan 3
hoofdstad 3
naaminlandstaal 3
locatie 3
functie4 3
valutacode 3
Vlag_Voor1 3
tld 3
lengte 3
feestdag 3
locatiekaart 3
politiek 3
Voor1 3
landcode 3
zieook 3
tijdzone 3
onderschrift2 2
tijdvak4 2
tijdvak1 2
tijdvak3 2
tijdvak2 2
regeringsleider 2
bijnaam 2
partij 2
stroming 2
functie5 2
oppervlakte 2
vlagartikel 2
afbeelding2 2
rang 2
titulatuur 2
soort 2
CIAkaart 2
wapenartikel 2
type 2
land 2
staatshoofd 2
euroorganisatie 1
onderschrift3 1
aantalmanen 1
zee 1
eurofractie 1
internationaal 1
hoofdkantoor 1
eMedicineSubj 1
afbeeldingonderschrift2 1
huidige functie 1
postcode 1
onderzoek2 1
ICD9 1
tijdvak5 1
tijdvak7 1
tijdvak6 1
pro

In [10]:
def sort(attributes):
    start_time = time.time()
    less = {}
    equal = {}
    greater = {}

    if len(attributes) > 1:
        pivot = attributes.iterkeys().next()
        for attr in attributes:
            if attributes[attr]['occurrence'] < pivot:
                less[attr] = attributes[attr]
            if attributes[attr]['occurrence'] == pivot:
                equal[attr] = attributes[attr]
            if attributes[attr]['occurrence'] > pivot:
                greater[attr] = attributes[attr]
        # Don't forget to return something!
        newDict = less.copy()
        newDict.update(equal)
        newDict.update(greater)
        for key in newDict:
            print newDict[key]['occurrence']
        return sort(less)+equal+sort(greater)  # Just use the + operator to join lists
    # Note that you want equal ^^^^^ not pivot
    else:  # You need to hande the part at the end of the recursion - when you only have one element in your array, just return the array.
        return array

# print sort(attributes)

## Gemmideld aantal attributen per infobox

In [72]:
from __future__ import division

def averageAttributesPerPage():
    sortedAttributes = sorted(attributes.items(), key=operator.itemgetter(1), reverse=True)
    totalAttr = 0
    for attr in sortedAttributes:
        try:
            totalAttr += attr[1]['occurrence']
        except:
            pass
    print "Total number of attributes occurrences is ",str(totalAttr)
    print "Total number of infoboxes is ",str(len(importDict()))
    print "Average amount of attributes per page(with infobox) is ",round(totalAttr/len(importDict()), 2)
    
averageAttributesPerPage()

Total number of attributes occurrences is  463
Total number of infoboxes is  22
Average amount of attributes per page(with infobox) is  21.05


## Significante types gevonden in attributen

In [166]:
def findTypes():
    for attribuut in attributes:
        printLine = False
        types = []
        title = "Attribuut "+attribuut+" heeft types : "
        for prop in attributes[attribuut]:
            if prop != "occurrence" and prop != "":
                # check if occurence is significant(bigger then 1)
                if attributes[attribuut][prop] > 1:
                    printLine = True
                    types.append("- "+prop+" with occurrence "+str(attributes[attribuut][prop]))
        if printLine:
            print title
            for line in types:
                print line
            print "\n"
findTypes()

Attribuut oorsprong heeft types : 
- volle, rauwe geitenmelk with occurrence 3
- [[Verenigde Staten]] with occurrence 2
- [[Los Angeles]] with occurrence 2
- [[Stockholm]], [[Zweden]] with occurrence 2
- {{BE}} // [[Antwerpen (stad)|Antwerpen]] with occurrence 3
- [[Londen]], [[Engeland]] with occurrence 11
- [[Los Angeles]], [[Californië]], [[Verenigde Staten]] with occurrence 4
- gepasteuriseerde [[koemelk]] with occurrence 2
- [[Volendam]], [[Nederland]] with occurrence 2
- [[Amstel (rivier)|Amstel]] with occurrence 2
- [[Engeland]] with occurrence 2
- [[Glasgow]], [[Schotland]] with occurrence 3
- rauwe, volle [[geitenmelk]] with occurrence 2
- [[Frankrijk]] with occurrence 2
- koemelk with occurrence 33
- de [[Beskiden]] with occurrence 2
- [[Noorwegen]] with occurrence 2
- [[Amsterdam]], [[Nederland]] with occurrence 2
- [[Eindhoven]], [[Noord-Brabant]], [[Nederland]] with occurrence 2
- [[Plateau van Langres]] with occurrence 3
- [[IJ (water)|IJ]] with occurrence 4
- schapenmelk

## Duplicaten in attributen

In [ ]:
def findDuplicateAttributes():
    